In [1]:
import os
import sys
import json
from sklearn.model_selection import train_test_split
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.hooks import EarlyStoppingHook
from mmengine.visualization import Visualizer, WandbVisBackend
from mmdet.utils import setup_cache_size_limit_of_dynamo
import datetime
import wandb

# mmdetection을 시스템 경로에 추가
sys.path.insert(0, "../mmdetection")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 사용자 맞춤형 학습 설정
config_path = './projects/CO-DETR/configs/codino/co_dino_5scale_swin_l_16xb1_1x_coco.py'  # DINO 설정 파일 경로
work_dir = './work_dirs/co_dino_custom'  # 로그와 모델을 저장할 디렉토리 경로
train_data_root = '/data/ephemeral/home/dataset/'  # 학습 데이터 경로
original_ann_file = '/data/ephemeral/home/dataset/train.json'  # 전체 데이터 어노테이션 파일 경로

In [3]:
# 학습 및 검증 데이터셋 나누기
with open(original_ann_file, 'r') as f:
    annotations = json.load(f)

# 이미지 ID 리스트 가져오기
image_ids = [image['id'] for image in annotations['images']]

# 학습 및 검증 셋으로 분할 (80% 학습, 20% 검증)
train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

# 학습 및 검증 데이터 어노테이션 생성
train_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in train_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in train_ids],
    'categories': annotations['categories']
}
val_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in val_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in val_ids],
    'categories': annotations['categories']
}

# 분할된 어노테이션을 파일로 저장
train_ann_file = '/data/ephemeral/home/dataset/train_split.json'
val_ann_file = '/data/ephemeral/home/dataset/val_split.json'

with open(train_ann_file, 'w') as f:
    json.dump(train_annotations, f)
with open(val_ann_file, 'w') as f:
    json.dump(val_annotations, f)

In [4]:
# 어노테이션 파일 수정: category_id를 1부터 시작하도록 조정
def increment_category_id(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # 'categories' 섹션의 'id'를 1부터 시작하도록 수정
    id_mapping = {}
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id + 1
        id_mapping[old_id] = new_id
        category['id'] = new_id

    # 'annotations' 섹션의 'category_id'를 매핑에 따라 수정
    for ann in data['annotations']:
        old_cat_id = ann['category_id']
        if old_cat_id in id_mapping:
            ann['category_id'] = id_mapping[old_cat_id]
        else:
            print(f"Warning: annotation id {ann['id']} has invalid category_id {old_cat_id}")

    # 수정된 데이터를 원래 파일에 저장
    with open(annotation_file, 'w') as f:
        json.dump(data, f, indent=4)

# 어노테이션 파일에 category_id를 1부터 시작하도록 수정
increment_category_id(train_ann_file)
increment_category_id(val_ann_file)


In [5]:
# 클래스 설정
classes = (
    "General trash", "Paper", "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
)
num_classes = len(classes)  # 10

In [6]:
# 설정 파일 로드
cfg = Config.fromfile(config_path)

# 작업 디렉토리 수정
cfg.work_dir = work_dir  # 로그와 모델 저장을 위한 작업 디렉토리 수정

# EarlyStoppingHook 추가
early_stopping_hook = dict(
    type='EarlyStoppingHook',
    monitor='bbox_mAP',
    rule='greater',
    min_delta=0.001,
    patience=5,
    check_finite=True,
    stopping_threshold=None
)

# cfg.default_hooks에 EarlyStoppingHook 추가
cfg.default_hooks.update(
    early_stopping=early_stopping_hook
)

# WandB 초기화 (Runner 전에)
run_name = f'co_dino_5scale_swin_l_16xb1_1x_coco_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'
wandb.init(
    project='Object_detection',
    name=run_name,
    config=cfg.to_dict(),
    allow_val_change=True,
    reinit=True
)
print(f"WandB Run Initialized: {run_name}")

# WandbVisBackend 설정
wandb_vis_backend = dict(
    type='WandbVisBackend',
    save_dir=cfg.work_dir,  # 저장할 디렉토리
    init_kwargs=dict(
        project='Object_detection',  # WandB 프로젝트 이름
        name=run_name,                # 고유한 실행 이름
        allow_val_change=True         # 설정 값 변경 허용
    ),
    define_metric_cfg=None,
    commit=True,
    log_code_name=None,
    watch_kwargs=None
)

# Visualizer 설정을 Runner의 visualizer 필드로 추가
cfg.visualizer = dict(
    type='Visualizer',
    vis_backends=[
        wandb_vis_backend  # WandbVisBackend 추가
    ],
    name='visualizer'  # Visualizer 이름 (옵션)
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


WandB Run Initialized: co_dino_5scale_swin_l_16xb1_1x_coco_20241008_124328


In [7]:
# 데이터셋 루트 경로 수정
cfg.train_dataloader.dataset.ann_file = train_ann_file  # 학습 데이터 어노테이션 파일 경로 수정
cfg.train_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 학습 데이터 이미지 경로 수정

cfg.val_dataloader.dataset.ann_file = val_ann_file  # 검증 데이터 어노테이션 파일 경로 수정
cfg.val_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 검증 데이터 이미지 경로 수정

# 클래스 설정을 데이터셋의 metainfo에 추가
cfg.train_dataloader.dataset.metainfo = dict(classes=classes)  # 학습 데이터셋 클래스 설정
cfg.val_dataloader.dataset.metainfo = dict(classes=classes)    # 검증 데이터셋 클래스 설정

# 클래스 수 수정
# bbox_head가 리스트인 경우 모든 헤드에 대해 num_classes 설정
if isinstance(cfg.model.bbox_head, list):
    for head in cfg.model.bbox_head:
        head['num_classes'] = num_classes
else:
    cfg.model.bbox_head['num_classes'] = num_classes

# roi_head 내의 bbox_head도 설정
if isinstance(cfg.model.roi_head, list):
    for roi_head in cfg.model.roi_head:
        if 'bbox_head' in roi_head:
            roi_head['bbox_head']['num_classes'] = num_classes
else:
    if 'bbox_head' in cfg.model.roi_head:
        cfg.model.roi_head['bbox_head']['num_classes'] = num_classes

# query_head도 num_classes 설정
if isinstance(cfg.model.query_head, list):
    for q_head in cfg.model.query_head:
        q_head['num_classes'] = num_classes
else:
    cfg.model.query_head['num_classes'] = num_classes

In [8]:
# 추가: 모델의 모든 관련 헤드에 num_classes가 올바르게 설정되었는지 확인
def verify_num_classes(cfg, num_classes):
    # bbox_head
    if isinstance(cfg.model.bbox_head, list):
        for head in cfg.model.bbox_head:
            assert head.get('num_classes') == num_classes, "bbox_head num_classes 설정 오류"
    else:
        assert cfg.model.bbox_head.get('num_classes') == num_classes, "bbox_head num_classes 설정 오류"
    
    # roi_head.bbox_head
    if isinstance(cfg.model.roi_head, list):
        for roi_head in cfg.model.roi_head:
            if 'bbox_head' in roi_head:
                assert roi_head['bbox_head'].get('num_classes') == num_classes, "roi_head.bbox_head num_classes 설정 오류"
    else:
        if 'bbox_head' in cfg.model.roi_head:
            assert cfg.model.roi_head['bbox_head'].get('num_classes') == num_classes, "roi_head.bbox_head num_classes 설정 오류"
    
    # query_head
    if isinstance(cfg.model.query_head, list):
        for q_head in cfg.model.query_head:
            assert q_head.get('num_classes') == num_classes, "query_head num_classes 설정 오류"
    else:
        assert cfg.model.query_head.get('num_classes') == num_classes, "query_head num_classes 설정 오류"
    
    print("모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.")

verify_num_classes(cfg, num_classes)

모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.


In [9]:
# 자동 혼합 정밀도 학습 사용 설정
use_amp = False  # 자동 혼합 정밀도(AMP) 학습 사용 여부
if use_amp:
    cfg.optim_wrapper.type = 'AmpOptimWrapper'
    cfg.optim_wrapper.loss_scale = 'dynamic'

# 학습률 자동 스케일링 설정
auto_scale_lr = False  # 학습률 자동 스케일링 사용 여부
if auto_scale_lr:
    if 'auto_scale_lr' in cfg and 'enable' in cfg.auto_scale_lr and 'base_batch_size' in cfg.auto_scale_lr:
        cfg.auto_scale_lr.enable = True
    else:
        raise RuntimeError('설정 파일에 "auto_scale_lr" 또는 필요한 키가 없습니다.')


In [10]:
# 데이터 로더 설정 최적화
cfg.train_dataloader.batch_size = 1  # 배치 사이즈를 1로 줄임
cfg.val_dataloader.batch_size = 1

cfg.train_dataloader.num_workers = 2  # 워커 수 줄이기
cfg.val_dataloader.num_workers = 2

# Prefetch factor와 persistent_workers 설정
cfg.train_dataloader.prefetch_factor = 2
cfg.train_dataloader.persistent_workers = False
cfg.val_dataloader.prefetch_factor = 2
cfg.val_dataloader.persistent_workers = False

In [11]:
# 학습 재개 설정
resume_training = False  # 학습 재개 여부 설정 (True로 설정 시 학습 재개)
resume_checkpoint_path = None  # 학습 재개 시 체크포인트 경로 지정
if resume_training:
    if resume_checkpoint_path is None:
        cfg.resume = True
        cfg.load_from = None
    else:
        cfg.resume = True
        cfg.load_from = resume_checkpoint_path

In [12]:
# 반복적인 컴파일 횟수를 줄여 학습 속도 향상
setup_cache_size_limit_of_dynamo()

In [13]:
# 평가 설정 추가 및 수정
cfg.evaluation = dict(interval=1, metric='bbox', save_best='auto')

# 평가자(evaluator) 설정 수정
if hasattr(cfg, 'val_evaluator'):
    # 평가자가 딕셔너리 또는 리스트로 정의되어 있을 경우
    if isinstance(cfg.val_evaluator, dict):
        cfg.val_evaluator.ann_file = val_ann_file
    elif isinstance(cfg.val_evaluator, list):
        for evaluator in cfg.val_evaluator:
            evaluator['ann_file'] = val_ann_file
elif 'evaluation' in cfg:
    # 평가 섹션 내에 ann_file을 추가
    cfg.evaluation['ann_file'] = val_ann_file


In [15]:
# 설정 파일을 통해 러너 생성
runner = Runner.from_cfg(cfg)

# ---------------------- 레이어 동결 및 Unfreeze 설정 시작 ----------------------

# 1. Backbone의 frozen_stages는 이미 설정되어 있으므로 첫 번째 stage는 유지

# 2. 모든 레이어를 동결
for name, param in runner.model.named_parameters():
    param.requires_grad = False

# 3. 필요한 레이어 Unfreeze
# bbox_head Unfreeze
for param in runner.model.bbox_head.parameters():
    param.requires_grad = True

# backbone의 마지막 두 레이어 Unfreeze
for name, param in runner.model.backbone.named_parameters():
    if 'stage3' in name or 'stage4' in name:
        param.requires_grad = True

# query_head Unfreeze
for name, param in runner.model.query_head.named_parameters():
    param.requires_grad = True

# roi_head Unfreeze
for name, param in runner.model.roi_head.named_parameters():
    param.requires_grad = True

# neck Unfreeze
for name, param in runner.model.neck.named_parameters():
    param.requires_grad = True

# rpn_head Unfreeze
for name, param in runner.model.rpn_head.named_parameters():
    param.requires_grad = True

# 7. Optimizer 설정 수정
cfg.optim_wrapper.paramwise_cfg = dict(
    custom_keys={
        'backbone.stage3': dict(lr_mult=0.1),
        'backbone.stage4': dict(lr_mult=0.1),
        'neck': dict(lr_mult=1.0),
        'query_head': dict(lr_mult=1.0),
        'bbox_head': dict(lr_mult=1.0),
        'roi_head': dict(lr_mult=1.0),
        'rpn_head': dict(lr_mult=1.0)
    }
)

10/08 12:43:34 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1717224637
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-g

/data/ephemeral/home/baseline/mmdetection/../mmdetection/projects/CO-DETR/codetr/transformer.py:1325: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  warnings.warn('If you want to reduce GPU memory usage, \
/data/ephemeral/home/baseline/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


10/08 12:43:49 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.


KeyError: 'DebugHook is not in the mmdet::hook registry. Please check whether the value of `DebugHook` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'

In [14]:

# 학습 시작 전에 검증 어노테이션 파일 존재 여부 확인
if not os.path.exists(val_ann_file):
    raise FileNotFoundError(f'Validation annotation file not found: {val_ann_file}')

# 학습 시작
runner.train()

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
10/08 01:09:15 - mmengine - WARNING - backbone.patch_embed.projection.weight is skipped since its requires_grad=False
10/08 01:09:15 - mmengine - WARNING - backbone.patch_embed.projection.bias is skipped since its requires_grad=False
10/08 01:09:15 - mmengine - WARNING - backbone.patch_embed.norm.weight is skipped since its requires_grad=False
10/08 01:09:15 - mmengine - WARNING - backbone.patch_embed.norm.bias is skipped since its requires_grad=False
10/08 01:09:15 - mmengine - WARNING - backbone.stages.0.blocks.0.norm1.weight is skipped since its requires_grad=False
10/08 01:09:15 - mmengine - WARNING - backbone.stages.0.blocks.0.norm1.bias is skipped since its requires_grad=False
10/08 01:09:15 - mmengine - WARNING - backbone.stages.0.blocks.0.attn.w_msa.relative_position_bias_table is skipped since its requires_grad=False
10/08 01:09:15 - mmengine - WARNING - backbone.stages.0.blocks.0.attn.w_msa.qkv

/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/data/ephemeral/home/baseline/mmdetection/../mmdetection/projects/CO-DETR/codetr/transformer.py:991: UserWarning: __floordiv__ is deprecated, and its behavio

10/08 01:10:01 - mmengine - INFO - Epoch(train)   [1][  50/3906]  base_lr: 1.0000e-04 lr: 1.0000e-04  eta: 3 days, 18:37:47  time: 0.8354  data_time: 0.0111  memory: 10067  grad_norm: 188.2975  loss: 217.6434  loss_cls: 0.0738  loss_bbox: 5.7226  loss_iou: 2.5558  d0.loss_cls: 0.0902  d0.loss_bbox: 5.7326  d0.loss_iou: 2.5609  d1.loss_cls: 0.0832  d1.loss_bbox: 5.7299  d1.loss_iou: 2.5593  d2.loss_cls: 0.0647  d2.loss_bbox: 5.7183  d2.loss_iou: 2.5679  d3.loss_cls: 0.0609  d3.loss_bbox: 5.7139  d3.loss_iou: 2.5594  d4.loss_cls: 0.0647  d4.loss_bbox: 5.7188  d4.loss_iou: 2.5639  enc_loss_cls: 0.0950  enc_loss_bbox: 5.6921  enc_loss_iou: 2.5691  dn_loss_cls: 0.2357  dn_loss_bbox: 1.7760  dn_loss_iou: 1.4309  d0.dn_loss_cls: 0.2554  d0.dn_loss_bbox: 1.7327  d0.dn_loss_iou: 1.3439  d1.dn_loss_cls: 0.2182  d1.dn_loss_bbox: 1.7407  d1.dn_loss_iou: 1.3621  d2.dn_loss_cls: 0.2195  d2.dn_loss_bbox: 1.7496  d2.dn_loss_iou: 1.3830  d3.dn_loss_cls: 0.2308  d3.dn_loss_bbox: 1.7602  d3.dn_loss_iou: 

/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/data/ephemeral/home/baseline/mmdetection/../mmdetection/projects/CO-DETR/codetr/transformer.py:991: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='fl

10/08 02:01:21 - mmengine - INFO - Epoch(val)   [1][ 50/977]    eta: 0:03:54  time: 0.2534  data_time: 0.0053  memory: 9717  
10/08 02:01:33 - mmengine - INFO - Epoch(val)   [1][100/977]    eta: 0:03:41  time: 0.2509  data_time: 0.0025  memory: 2215  
10/08 02:01:46 - mmengine - INFO - Epoch(val)   [1][150/977]    eta: 0:03:28  time: 0.2508  data_time: 0.0027  memory: 2215  
10/08 02:01:58 - mmengine - INFO - Epoch(val)   [1][200/977]    eta: 0:03:15  time: 0.2513  data_time: 0.0024  memory: 2215  
10/08 02:02:11 - mmengine - INFO - Epoch(val)   [1][250/977]    eta: 0:03:02  time: 0.2513  data_time: 0.0026  memory: 2215  
10/08 02:02:23 - mmengine - INFO - Epoch(val)   [1][300/977]    eta: 0:02:50  time: 0.2508  data_time: 0.0024  memory: 2215  
10/08 02:02:36 - mmengine - INFO - Epoch(val)   [1][350/977]    eta: 0:02:37  time: 0.2511  data_time: 0.0026  memory: 2215  
10/08 02:02:48 - mmengine - INFO - Epoch(val)   [1][400/977]    eta: 0:02:25  time: 0.2507  data_time: 0.0024  memory:

...

10/08 05:51:28 - mmengine - INFO - Epoch(train)   [6][  50/3906]  base_lr: 1.0000e-04 lr: 1.0000e-04  eta: 3 days, 9:57:21  time: 0.8120  data_time: 0.0126  memory: 10060  grad_norm: 142.6973  loss: 70.3455  loss_cls: 0.6491  loss_bbox: 0.2119  loss_iou: 0.2907  d0.loss_cls: 0.6355  d0.loss_bbox: 0.3287  d0.loss_iou: 0.3907  d1.loss_cls: 0.6229  d1.loss_bbox: 0.2632  d1.loss_iou: 0.3499  d2.loss_cls: 0.6413  d2.loss_bbox: 0.2296  d2.loss_iou: 0.3120  d3.loss_cls: 0.6434  d3.loss_bbox: 0.2179  d3.loss_iou: 0.2963  d4.loss_cls: 0.6512  d4.loss_bbox: 0.2114  d4.loss_iou: 0.2869  enc_loss_cls: 0.7665  enc_loss_bbox: 0.3837  enc_loss_iou: 0.4513  dn_loss_cls: 0.0892  dn_loss_bbox: 0.4415  dn_loss_iou: 0.4467  d0.dn_loss_cls: 0.1002  d0.dn_loss_bbox: 0.6785  d0.dn_loss_iou: 0.6664  d1.dn_loss_cls: 0.0845  d1.dn_loss_bbox: 0.5166  d1.dn_loss_iou: 0.5273  d2.dn_loss_cls: 0.0855  d2.dn_loss_bbox: 0.4673  d2.dn_loss_iou: 0.4740  d3.dn_loss_cls: 0.0865  d3.dn_loss_bbox: 0.4479  d3.dn_loss_iou: 0.

/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/data/ephemeral/home/baseline/mmdetection/../mmdetection/projects/CO-DETR/codetr/transformer.py:991: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='fl

10/08 06:43:06 - mmengine - INFO - Epoch(val)   [6][ 50/977]    eta: 0:03:56  time: 0.2552  data_time: 0.0067  memory: 9714  
10/08 06:43:19 - mmengine - INFO - Epoch(val)   [6][100/977]    eta: 0:03:42  time: 0.2515  data_time: 0.0027  memory: 2215  
10/08 06:43:31 - mmengine - INFO - Epoch(val)   [6][150/977]    eta: 0:03:28  time: 0.2504  data_time: 0.0023  memory: 2215  
10/08 06:43:44 - mmengine - INFO - Epoch(val)   [6][200/977]    eta: 0:03:15  time: 0.2506  data_time: 0.0023  memory: 2215  
10/08 06:43:56 - mmengine - INFO - Epoch(val)   [6][250/977]    eta: 0:03:02  time: 0.2503  data_time: 0.0023  memory: 2215  
10/08 06:44:09 - mmengine - INFO - Epoch(val)   [6][300/977]    eta: 0:02:50  time: 0.2508  data_time: 0.0023  memory: 2215  
10/08 06:44:22 - mmengine - INFO - Epoch(val)   [6][350/977]    eta: 0:02:37  time: 0.2518  data_time: 0.0025  memory: 2215  
10/08 06:44:34 - mmengine - INFO - Epoch(val)   [6][400/977]    eta: 0:02:25  time: 0.2508  data_time: 0.0024  memory:

IndexError: list index out of range

: 